In [1]:
import onnx
import onnxruntime

onnx_model = onnx.load("./saved_models/recognitionModel.onnx")
try:
    onnx.checker.check_model(onnx_model)
except onnx.checker.ValidationError as e:
    print('The model is invalid: %s' % e)
else:
    print('The model is valid!')

The model is valid!


In [2]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [6]:
import onnxruntime
import torch
from PIL import Image
from utils import NormalizePAD, adjust_contrast_grey
import numpy as np
import math

image = Image.open("all_data/eval_img/7338.jpg")

from utils import reformat_input
img, img_cv_gray = reformat_input(image)
image = Image.fromarray(img_cv_gray, 'L')

transform = NormalizePAD((1, 32, 100))

w, h = image.size
adjust_contrast = 0.
#### augmentation here - change contrast
if adjust_contrast > 0:
    image = np.array(img_cv_gray.convert("L"))
    image = adjust_contrast_grey(image, target = adjust_contrast)
    image = Image.fromarray(image, 'L')

ratio = w / float(h)
if math.ceil(32 * ratio) > 100:
    resized_w = 100
else:
    resized_w = math.ceil(32 * ratio)
image = image.resize((resized_w, 32), Image.BICUBIC)
image = transform(image)
image

tensor([[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [1.0000, 1.0000, 0.9922,  ..., 0.9843, 1.0000, 1.0000],
         [1.0000, 1.0000, 0.9922,  ..., 0.9765, 1.0000, 1.0000],
         [1.0000, 1.0000, 0.9922,  ..., 0.9843, 1.0000, 1.0000]]])

In [7]:
ort_session = onnxruntime.InferenceSession("./saved_models/recognitionModel.onnx")
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(image)}
ort_outs = ort_session.run(None, ort_inputs)
preds = torch.from_numpy(ort_outs[0])

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Invalid rank for input: images_graycay Got: 3 Expected: 4 Please fix either the inputs or the model.